PyTorch를 사용하면서 문제가 생길 때의 해결법

공포의 단어 : OOM(out of memory)

### OOM을 해결하기 어려운 이유
- 왜 발생했는지 알기 어려움
- 어디서 발생했는지 알기 어려움  
    : Python과 다르게 PyTorch는 발생 위치를 말해주지 않는다.
- Error backtracking이 이상한데로 감
- memory의 이전상황의 파악이 어려움  
    : 보통 error는 iteration이 돌아가면서 발생

#### 단순한 해결방법 : Batch Size를 줄이고 GPU를 clean하고 다시 실행

### 그 외 발생할 만한 문제들

### GPU Util 사용하기
- 현재 GPU의 상태를 정확히 알기 위해 사용한다.
- nvidia-smi 처럼 GPU의 상태를 보여주는 모듈
- Colab은 환경에서 GPU 상태를 보여주기 편함
- iter마다 메모리가 늘어나는지 확인

### torch.cuda.empty_cache() 써보기
- 사용되지 않은 GPU상 cache를 정리
- 가용 메모리를 확보
- del과는 구분이 필요(del은 관계를 끊어버린다)
- reset 대신 쓰기 좋은 함수

### trainnin loop에 tensor로 축적되는 변수는 확인할 것
- tensor로 처리된 변수는 GPU상에 메모리 사용
- 해당 변수 loop안에 연산에 있을 때 GPU에 conputational graph를 생성(메모리 잠식)
- 1-d tensor의 경우 python 기본 객체로 변환하여 처리할 것


### del 명령어를 적절히 사용하기
- 필요가 없어진 변수는 적절한 삭제가 필요
- python의 메모리 배치 특성상 loop이 끝나도 메모리를 차지함

In [3]:
for x in range(10):
    i = x
print(i) # iter가 끝나도 i가 남아 있다

9


### 가능 batch 사이즈 실험해보기
- 학습시 OOM이 발생했다면 batch 사이즈를 1로 해서 실험해보기

### torch.no_grad() 사용하기
- Inference 시점에서는 torch.no_grad() 구문을 사용
- backward pass 으로 인해 쌓이는 메모리에서 자유로움
- 습관적으로 Inference 시점에서는 사용하는게 좋다

### 예상치 못한 에러 메시지
- OOM 말고도 유사한 에러가 발생
- CUDNN_STATUS_NOT_INIT 이나 device-side-assert 등
- 해당 에러도 cuda와 관련하여 OOM의 일종이므로, 적절한 코드 처리 필요

#### 그외
- colab에서는 너무 큰 사이즈는 실행하지말 것(linear, CNN, LSTM)
- CNN의 대부분의 에러는 크기가 안맞아서 생김(torchsummary등으로 사이즈를 맞출것)
- tensor의 float precision을 16bit로 줄일 수 있다.